# Captain Freako's Solutions to *Typeclassopedia* Exercises
Original author: David Banas <capn.freako@gmail.com>  
Original date: September 25, 2015

Copyright (c) 2015 David Banas; all rights reserved World wide.

This [jupyter notebook](https://github.com/gibiansky/ihaskell) contains my solutions to some of the exercises posed by [Brent Yorgey](mailto:byorgey@cis.upenn.edu), in his [Typeclassopedia](https://wiki.haskell.org/Typeclassopedia) document.

## Additional Contributors
- Phil Ruffwind <rf@rufflewind.com>


## Contents
- <a href="#3.2">Section 3.2 - Functor Instances</a>
    - <a href="#3.2.1">Ex. 3.2.1 - Functor instances for Either e and ((->) e)</a>
    - <a href="#3.2.2">Ex. 3.2.2 - Functor instances for ((,) e) and Pair</a>
    - <a href="#3.2.3">Ex. 3.2.3 - Functor instances for a tree</a>
    - <a href="#3.2.4">Ex. 3.2.4 - Non-Functor type of kind: \* -> \* </a>
    - <a href="#3.2.5">Ex. 3.2.5 - Is the composition of two Functors also a Functor?</a>
- <a href="#3.3">Section 3.3 - Functor Laws</a>
    - <a href="#3.3.1">Ex. 3.3.1 - Satisfies second Functor law, but not first.</a>
    - <a href="#3.3.2">Ex. 3.3.2 - Which laws violated by bad Functor list instance?</a>
- <a href="#4.2">Section 4.2 - Applicative Laws</a>
    - <a href="#4.2.1">Ex. 4.2.1 - Applying a pure function to an effectful argument.</a>


## Section 3.2 - Functor Instances <a name="3.2"/>

### Ex. 3.2.1 - Functor instances for Either e and ((->) e) <a name="3.2.1"/>

In [ ]:
class MyFunctor f where
    fmap' :: (a -> b) -> f a -> f b

instance MyFunctor (Either e) where
    fmap' g (Left e)  = Left e
    fmap' g (Right x) = Right (g x)

instance MyFunctor ((->) e) where
    fmap' g f = g . f

-- Testing Functor law on defined instances.
func_law x = fmap' id x == x  -- "id x" was producing an annoying error.

func_law2 x = fmap' id f x == f x
  where
    f x = 2 * x

func_law (Left "Error String")
func_law (Right 1)
func_law2 (2::Int)


### Ex. 3.2.2 - Functor instances for ((,) e) and Pair <a name="3.2.2"/>

In [ ]:
instance MyFunctor ((,) e) where
    fmap' g (e, a) = (e, g a)

data Pair a = Pair a a deriving (Eq, Show)

instance MyFunctor Pair where
    fmap' g (Pair x y) = Pair (g x) (g y)

func_law ("msg", 0.1)
func_law $ Pair 1 2


__Discussion, as per exercise instructions:__

  -  Similarities:
  
     - Both types/instances return a pair of values.

  -  Differences:
  
    - The type of the first member of the pair can differ from that
      of the second, in the '(,) e' case.
      
    - The mapped function, g, is only mapped over the second member
      of the pair, in the '(,) e' case.
      
    - The type of the second member of the pair is NOT included in
      the Functor instance, for either type, but the type of the
      first member IS included, for ((,) e), but NOT for Pair.


### Ex. 3.2.3 - Functor instance for a tree <a name="3.2.3"/>

In [ ]:
data ITree a = Leaf (Int -> a) 
             | Node [ITree a]

instance Eq a => Eq (ITree a) where
    Leaf g == Leaf h = g 0 == h 0
    Node xs == Node ys = and (zipWith (==) xs ys)
    Leaf _ == Node _ = False
    Node _ == Leaf _ = False

instance MyFunctor ITree where
    fmap' g (Leaf f)   = Leaf (g . f)
    fmap' g (Node its) = Node $ map (fmap' g) its

-- Test Functor identity law.
let f = Node [Leaf id, Leaf id] in
  fmap' id f == f


### Ex. 3.2.4 - Non-Functor type of kind: \* -> \* <a name="3.2.4"/>

In [ ]:
{-# LANGUAGE DeriveFunctor DatatypeContexts #-}

-- data NoFunc a = NoFunc a deriving (Functor)  -- Compiles without error.
data (Ord a) => NoFunc a = NoFunc a deriving (Functor)


### Ex. 3.2.5 - Is the composition of two Functors also a Functor? <a name="3.2.5"/>

In [ ]:
-- Yes: The following code compiles without error.

{-# LANGUAGE DeriveFunctor #-}

data Func1 a = Func1 a deriving (Functor)
data Func2 b = Func2 b deriving (Functor)
data CompFunc b = CompFunc (Func1 (Func2 b)) deriving (Functor)


## Section 3.3 - Functor Laws <a name="3.3"/>

### Ex. 3.3.1 - Satisfies second Functor law, but not first. <a name="3.3.1"/>
I was pointed to the following solution, by Phil Ruffwind <rf@rufflewind.com>:

[Stack Overflow Solution](http://stackoverflow.com/questions/13539335/haskell-first-functor-law-from-second/13539433#13539433)


In [ ]:
data Break a = Yes | No deriving (Eq)

instance MyFunctor Break where
   fmap' f _ = No

-- Test Functor id law.
fmap' id Yes == Yes

-- Test Functor composition law.
let g = id
    h = const Yes
  in fmap' (g . h) Yes == (fmap' g . fmap' h) Yes


### Ex. 3.3.2 - Which laws violated by bad Functor list instance? <a name="3.3.2"/>
Both, because each application of fmap doubles the size of the list.

In [ ]:
-- Evil Functor instance
instance MyFunctor [] where
  fmap' _ [] = []
  fmap' g (x:xs) = g x : g x : fmap g xs

-- Testing Functor identity law.
fmap' id [1,2,3] == [1,2,3]

-- Testing Functor composition law.
fmap' ((+ 1) . (* 2)) [1,2,3] == (fmap' (+ 1) . fmap' (* 2)) [1,2,3]


## Section 4.2 - Applicative Laws <a name="4.2"/>

### Ex. 4.2.1 - Applying a pure function to an effectful argument. <a name="4.2.1"/>

__To prove:__

    pure f <*> x = pure (flip ($)) <*> x <*> pure f
    
__Available Properties:__

    pure f <*> x      = fmap f x                                    (1st Applicative Law)
    f <*> pure x      = pure ($ x) <*> f = pure (flip ($) x) <*> f  (interchange)
    pure f <*> pure x = pure (f x)                                  (homomorphism)

__Useful Relationships:__

    f     = \x -> f x =      ($) f = (f $)
    ($ x) = \f -> f x = flip ($) x
    
__Proof:__

    pure f <*> x = pure (\x -> f x) <*> x = pure (flip ($) (\x -> (\f -> f x))) <*> x
    
    pure f <*> pure (sequence x)
    
    f <$> x = pure ($ f) <*> x = (interchange)

    pure (flip ($)) <*> x <*> pure f        =  (homomorphism)
    pure flip <*> pure ($) <*> x <*> pure f =  (1st Applicative Law)
    fmap flip (pure ($)) <*> x <*> pure f


pure (flip ($)) <*> x <*> pure f     = (interchange)
x <*> pure ($ flip ($)) <*> pure f   =
x <*> pure ($ flip ($) f)

pure (flip ($)) <*> pure ($ f) <*> x = (homomorphism)
pure (flip ($) ($ f)) <*> x          = (definition of flip)
pure ($ ($ f)) <*> x                 = (interchange)
x <*> pure ($ f)                     = (interchange) (Is this step valid?)
    pure f <*> x

In [2]:
x = Just 1
:t x
x
y = (+ 1) <$> x
:t y
y


x :: forall a. Num a => Maybe a

Just 1

y :: forall b. Num b => Maybe b

Just 2

## Section 4.3 - Applicative Instances <a name="4.3"/>

### Ex. 4.3.1 - Applicative instance for Maybe <a name="4.3.1"/>

In [ ]:
class MyApplicative f where
    pure'  :: a -> f a
    (<@>)  :: f (a -> b) -> f a -> f b
    
instance MyApplicative Maybe where
    pure' = Just
    Nothing <@> _ = Nothing
    _ <@> Nothing = Nothing
    Just g <@> Just a = Just (g a)
    
-- Test the Applicative laws.

-- Identity
Just id <@> Nothing == Nothing
-- Curiously, even though the error message, below, calls for this,
-- when I attempt to use it I get an error saying the function
-- 'isNothing' is not in scope! (?!)
-- isNothing (Just id <@> Nothing)
Just id <@> Just 3  == Just 3

-- Homomorphism
Just (+ 2) <@> Just 3 == Just ((+ 2) 3)

-- Interchange
Just (+ 2) <@> Just 3 == Just ($ 3) <@> Just (+ 2)

-- Composition
Just (+ 2) <@> (Just (+ 3) <@> Just 4) == Just (.) <@> Just (+ 2) <@> Just (+ 3) <@> Just 4


### Ex. 4.3.2 - Correct Applicative instance for ZipList <a name="4.3.2"/>

In [ ]:
newtype ZipList a = ZipList { getZipList :: [a] } deriving (Eq)
 
instance MyApplicative ZipList where
  pure' x = ZipList (repeat x)
  (ZipList gs) <@> (ZipList xs) = ZipList (zipWith ($) gs xs)

-- Test the Applicative laws.

-- Identity
pure' id <@> ZipList ([1,2,3] :: [Int]) == ZipList [1,2,3]

-- Homomorphism
pure' (+ 2) <@> ZipList ([1,2,3] :: [Int]) == ZipList (map (+ 2) [1,2,3])

-- Interchange
ZipList (replicate 3 (+ 2)) <@> pure' 1 == pure' ($ 1) <@> ZipList (replicate 3 (+ 2))

-- Composition
ZipList (replicate 3 (+ 2)) <@> (ZipList (replicate 3 (+ 3)) <@> ZipList [1,2,3]) ==
    pure' (.) <@> ZipList (replicate 3 (+ 2)) <@> ZipList (replicate 3 (+ 3)) <@> ZipList [1,2,3]
